In [ ]:
import pandas as pd
import numpy as np
!pip install stanza
import stanza
from sklearn.feature_extraction.text import TfidfVectorizer
import re
!pip install bs4
from bs4 import BeautifulSoup
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct
!pip install contractions
import contractions
#!pip uninstall transformersy
!pip install --no-cache-dir transformers sentencepiece
from transformers import pipeline
import re
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import itertools
import copy
from transformers import AutoTokenizer, AutoModelForMaskedLM
!pip install -U sentence-transformers
!pip install detoxify
!pip install --upgrade git+https://github.com/flairNLP/flair.git
import pandas as pd
from detoxify import Detoxify
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from flair.embeddings import FlairEmbeddings


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-1et4mvjj
  Running command git clone -q

Read Bad words from Stanford Dataset

In [ ]:
with open('bad_word.txt') as f:
    bad_words =[line.rstrip("\n") for line in f.readlines()]

Reading the Reddit Dataset

In [ ]:
df = pd.read_csv('labeled_comments.csv',  error_bad_lines=False)
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,comments,toxicity_score,toxicity_classification
0,I have never met such a dumb fucking person li...,0.8000,1
1,You are a bloody asshole leader and I hate you.,0.8000,1
2,I have never met such a stupid and foolish per...,0.0002,0
3,You are a bitch. I do not like you.,0.8000,1
4,Never in my life have a met such a foolish per...,0.0020,0


Make Our Labels based on the toxicity score

In [ ]:
label =[]
for i in range(len(df)):
  if df.toxicity_score[i]>0.5:
    label.append(1)
  else:
    label.append(0)
df['label'] = label
df.head()

,comments,toxicity_score,toxicity_classification,label
0,I have never met such a dumb fucking person li...,0.8000,1,1
1,You are a bloody asshole leader and I hate you.,0.8000,1,1
2,I have never met such a stupid and foolish per...,0.0002,0,0
3,You are a bitch. I do not like you.,0.8000,1,1
4,Never in my life have a met such a foolish per...,0.0020,0,0


In [ ]:
# get only sentence 1 for good sentences
for i in range(len(df)):
  if df.label[i]==0:
    df['comments'][i] = df['comments'][i].split(".")[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
def text_prepare(text):
    #removing urls
    text = re.sub(r'http\S+', '', text)
    text = BeautifulSoup(text).get_text()
    #removing emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = contractions.fix(text)
    #removing email
    text =re.sub(r'[A-Za-z0-9]@[A-Za-z]\.?[A-Za-z0-9]*', "", text)
    #Keeping only alphabets
    regex = re.compile('[^a-zA-Z ]')
    text = regex.sub(' ',str(text))
    text = re.sub(' +', ' ', text)
    text = text.lower()
    # delete stopwords from text
    # text = ' '.join([word for word in text.split()]) 
    # text = text.strip()
    return text

In [ ]:
# Data cleaning
df['comments']= df['comments'].apply(lambda x:text_prepare(x))

In [ ]:
# getting good sentences
good_sentences = df[df.label==0][10000:12500].reset_index().drop(['index', 'toxicity_classification', 'toxicity_score', 'label'], axis=1)

Retrive- POS Tagging, searching

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


In [ ]:
def retrive_pos_tagging(balanced_dataset):
  pos = []
  pos_dict = []
  for i in range(len(balanced_dataset)):
    doc = nlp(balanced_dataset.comments[i])
    temp = []
    temp_dict = {}
    for sent in doc.sentences:
      for word in sent.words:
        if word.text.lower() in bad_words:
          temp.append('BW')
          if 'BW' in temp_dict:
            temp_dict['BW'].append(word.text)
          else:
            temp_dict['BW'] = [word.text]
        else:
          temp.append(word.xpos)
          if word.xpos in temp_dict:
            temp_dict[word.xpos].append(word.text)
          else:
            temp_dict[word.xpos] = [word.text]
    pos.append(temp)
    pos_dict.append(temp_dict)
  return pos, pos_dict

TFIDF Search

In [ ]:
pos, pos_dict = retrive_pos_tagging(good_sentences)
good_sentences['pos'] = pos
good_sentences['pos'] = good_sentences['pos'].str.join(" ")
good_sentences['pos_dict'] = pos_dict

In [ ]:
good_sentences

,comments,pos,pos_dict
0,o really,RB RB,"{'RB': ['o', 'really']}"
1,exactly,RB,{'RB': ['exactly']}
2,are you suggesting that the democrats plan to ...,VBP PRP VBG IN DT NNPS VBP TO VB NN VBZ NN NNS...,"{'VBP': ['are', 'plan'], 'PRP': ['you'], 'VBG'..."
3,trump s tax returns would be so inconsequentia...,NN POS NN NNS MD VB RB JJ DT NN NN PRP VBP RB ...,"{'NN': ['trump', 'tax', 'midterm', 'election']..."
4,how have they not weighed the issue already,WRB VBP PRP RB VBN DT NN RB,"{'WRB': ['how'], 'VBP': ['have'], 'PRP': ['the..."
...,...,...,...
2495,he does not have the authority,PRP VBZ RB VB DT NN,"{'PRP': ['he'], 'VBZ': ['does'], 'RB': ['not']..."
2496,you just know the next republican administrati...,PRP RB VBP DT JJ BW NN IN DT NN NN MD VB DT RB...,"{'PRP': ['you'], 'RB': ['just', 'very', 'unapo..."
2497,it is big government republican judges once ag...,PRP VBZ JJ NN BW NNS IN RB VBG DT NN IN DT NN ...,"{'PRP': ['it'], 'VBZ': ['is', 'is'], 'JJ': ['b..."
2498,what about a week before the election,WP IN DT NN IN DT NN,"{'WP': ['what'], 'IN': ['about', 'before'], 'D..."


In [ ]:
good_sentences.to_csv("/content/dus_to_baara.csv")

In [ ]:
good_sentences = pd.read_csv("good_sentences.csv")

In [ ]:
good_sentences.dropna(inplace = True)

In [ ]:
def get_tfidf_vectorizer(good_sentences, bad_sentences):
  balanced_dataset = pd.concat([good_sentences, bad_sentences], axis=0, ignore_index=True)
  tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.9, min_df=5, token_pattern='(\S+)')
  tf_idf_matrix = tfidf_vectorizer.fit(balanced_dataset['pos'])
  tf_idf_matrix_good = tfidf_vectorizer.transform(good_sentences['pos'])
  return tfidf_vectorizer, tf_idf_matrix_good

In [ ]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)
    matches = csr_matrix((data,indices,indptr),shape=(M,N)).toarray()
    ans =[]
    for i in range(len(matches)):
      a = matches[i]
      ans.append(np.argwhere(a>0))
    return ans

In [ ]:
def find_similar_good_sentences(bad_sentence, good_sentences, ans):
  similar_df= {'bad':[], 'good':[], 'bad_pos':[], 'good_pos':[], 'bad_pos_map':[], 'good_pos_map':[]}  
  similar_df['bad'].append(bad_sentence.comments[0])
  similar_df['bad_pos'].append(bad_sentence.pos[0])
  similar_df['bad_pos_map'].append(bad_sentence.pos_dict[0])
  similar_good = []
  similar_pos =[]
  similar_pos_map = []
  for i in ans:
    similar_good.append(good_sentences.comments[i[0]])
    similar_pos.append(good_sentences.pos.iloc[i[0]])
    similar_pos_map.append(good_sentences.pos_dict[i[0]])
  similar_df['good'].append(similar_good)
  similar_df['good_pos'].append(similar_pos)
  similar_df['good_pos_map'].append(similar_pos_map)
  return similar_df

Generate

In [ ]:
def make_assignments(list_1,list_2):
  unique_combinations = []
  # permut = itertools.permutations(list_1, len(list_2))
  # print(permut)

  if len(list_1)<len(list_2):
      permut = itertools.permutations(list_2, len(list_1))
      for comb in permut:
        zipped = zip(list_1,comb)
        unique_combinations.append(list(zipped))

  else:
    permut = itertools.permutations(list_1, len(list_2))
    for comb in permut:
        zipped = zip(comb,list_2)
        unique_combinations.append(list(zipped))
  
  return unique_combinations  
def generate(xi, pi, pi_dash, bad_words, pi_map, pi_dash_map):
  ti = set(pi)
  ti_dash = set(pi_dash)
  t_shared = ti.intersection(ti_dash)
  c0 = ["<mask>"]*len(pi_dash)
  ci = [c0]
  for inx, tk in enumerate(t_shared):
    if tk=='BW':
      continue
    wk = set(pi_map[tk])
    sk = [i for i, x in enumerate(pi_dash) if x == tk]
    assignments = make_assignments(wk, sk)
    if len(assignments)==0:
      continue
    new_ci = []
    for assignment in assignments:
      for prev_sent in ci:
        for word, indx in assignment:
          temp = copy.deepcopy(prev_sent)
          temp[indx]= word
        new_ci.append(temp)
    ci = new_ci
  return ci

In [ ]:
def fill_mask_word_roberta(input_string, classifier_roberta, puncs):
    if input_string.split(" ").count('<mask>') ==0:
       return input_string.split("[SEP]",1)[1].strip() 
    result = classifier_roberta(input_string)
    ans = input_string
    if ans.split(" ").count('<mask>') ==1:
       result = [result]
    for each_mask in result:
        for i in range(len(each_mask)):
            filler_word = each_mask[i]['token_str'].strip()
            if filler_word.lower() not in bad_words and filler_word not in puncs:
                ans = re.sub('<mask>', filler_word.lower(), ans, count=1)
                break
            if i == len(each_mask) - 1:
                ans = re.sub('<mask>', '', ans, count=1)
                break 
    return ans.split("[SEP]",1)[1].strip()


In [ ]:
def remove_adjacent(seq): # works on any sequence, not just on numbers
  i = 1
  n = len(seq)
  while i < n: # avoid calling len(seq) each time around
    if seq[i] == seq[i-1]:
      del seq[i]
      # value returned by seq.pop(i) is ignored; slower than del seq[i]
      n -= 1
    else:
      i += 1

In [ ]:
def embeddings_cosine_toxicity_perplexity(toxic_model, embedding_model, perplex_model, perplex_tokenizer, original, created_list):
  
  # Here we shall first make the embeddings of the sentences
  original_sent_embeddings = embedding_model.encode(original)
  created_sent_embeddings = embedding_model.encode(created_list)
  my_selection = {"original_sentence":original,"generated_sentences":created_list, "cosine_score":[], "non-toxicity_score":[], "perplexity_score":[]}
  cosine_calc = []
  toxic_calc = []
  perplex_calc = []

  # Now we will calclate the cosine with the original sentences
  for indx, i in enumerate(created_sent_embeddings):
    print(created_list[indx])
    toxic_calc.append(1-(toxic_model.predict(created_list[indx])['toxicity']))
    perplex_encodings = perplex_tokenizer(created_list[indx], return_tensors="pt")
    perplex_calc.append(gpt2_perplexity(perplex_encodings, perplex_model))
    # perplex_calc.append(perplex_model.calculate_perplexity(created_list[indx]))
    cosine_calc.append(cosine_similarity(original_sent_embeddings.reshape(1,-1), i.reshape(1,-1)).astype(float)[0][0])
  my_selection.update({"cosine_score": cosine_calc})
  my_selection.update({"non-toxicity_score":toxic_calc})
  my_selection.update({"perplexity_score":perplex_calc})
  
  return my_selection

In [ ]:

# tokenizer = AutoTokenizer.from_pretrained("conjuring92/distilroberta-base-finetuned-toxic")

model = AutoModelForMaskedLM.from_pretrained("conjuring92/distilroberta-base-finetuned-toxic")

# def get_prediction (sent):
    
#     token_ids = tokenizer.encode(sent, return_tensors='pt')
#     masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
#     masked_pos = [mask.item() for mask in masked_position ]

#     with torch.no_grad():
#         output = model(token_ids)

#     last_hidden_state = output[0].squeeze()

#     list_of_list =[]
#     for index,mask_index in enumerate(masked_pos):
#         mask_hidden_state = last_hidden_state[mask_index]
#         idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
#         words = [tokenizer.decode(i.item()).strip() for i in idx]
#         list_of_list.append(words)
#         print ("Mask ",index+1,"Guesses : ",words)
    
#     best_guess = ""
#     for j in list_of_list:
#         best_guess = best_guess+" "+j[0]
        
#     return best_guess

# sentence = " ".join(ci[3])
# print ("Original Sentence replaced with mask: ",sentence)
# print ("\n")

# predicted_blanks = get_prediction(sentence)
# print ("\nBest guess for fill in the blank :::",predicted_blanks)

Downloading:   0%|          | 0.00/693 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [ ]:
import torch
from tqdm import tqdm
import pandas as pd

In [ ]:
def gpt2_perplexity(sent_encoding, model):
    max_length = model.config.n_positions
    stride = 1
    seq_len = sent_encoding.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc
        input_ids = sent_encoding.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids[0], labels=target_ids[0])
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    return ppl.item()

In [ ]:
def get_civil_sentence(bad_sentence, good_sentences):
  bad_sentence = text_prepare(bad_sentence)
  bs = {'comments': bad_sentence}
  bs = pd.DataFrame([bs])
  # RETRIVE
  pos, pos_dict = retrive_pos_tagging(bs)
  bs['pos'] = ' '.join(pos[0])
  bs['pos_dict'] = pos_dict
  print(bs.comments[0], pos)
  tfidf_vectorizer, tf_idf_matrix_good = get_tfidf_vectorizer(good_sentences, bs) 
  tf_idf_matrix_bad = tfidf_vectorizer.transform(bs['pos'])
  ans = awesome_cossim_top(tf_idf_matrix_bad, tf_idf_matrix_good.transpose(), 10, 0.02)
  similar_df = pd.DataFrame(find_similar_good_sentences(bs, good_sentences, ans[0]))
  # GENERATE
  ci = []
  for j in range(0,10):
    b = len(similar_df.bad[0].split(" "))
    g = len(similar_df.good[0][j].split(" "))
    if g<=b+5 and g>=b-5:
      ci+=generate(similar_df.bad[0], similar_df.bad_pos[0].split(), similar_df.good_pos[0][j].split(), bad_words, similar_df.bad_pos_map[0], similar_df.good_pos_map[0][j])
  # fill masks
  classifier_roberta = pipeline('fill-mask', model="conjuring92/distilroberta-base-finetuned-toxic")
  puncs = ['.', ',']
  civil_sentences= []
  for c in ci:
    out = fill_mask_word_roberta(similar_df.bad[0]+"[SEP] "+" ".join(c), classifier_roberta, puncs).split(" ")
    remove_adjacent(out)
    civil_sentences.append(" ".join(out))
  print(ci)
  # SELECTION
  toxic_model = Detoxify('original')
  embedding_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
  perplex_model = FlairEmbeddings('news-forward').lm
  from transformers import GPT2LMHeadModel, GPT2TokenizerFast
  model_id = "gpt2-large"
  perplex_model = GPT2LMHeadModel.from_pretrained(model_id)
  perplex_model.eval()
  perplex_tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
  original_sent = similar_df.bad[0]
  created_sent = civil_sentences

  # Running my func
  my_caclulations = embeddings_cosine_toxicity_perplexity(toxic_model, embedding_model, perplex_model, perplex_tokenizer, original_sent, created_sent)

  # Printing
  my_calculations_df = pd.DataFrame.from_dict(my_caclulations)
  # my_calculations_df["cosine_score_w"] = my_calculations_df["cosine_score"]*0.50
  # my_calculations_df["non-toxicity_score_w"] = my_calculations_df["non-toxicity_score"]*0.20
  # my_calculations_df["perplexity_score_w"] = (my_calculations_df["perplexity_score"]*-1)*0.30
  # cols = ["cosine_score_w","non-toxicity_score_w","perplexity_score_w"]
  # my_calculations_df["sum_factor"] = my_calculations_df[cols].sum(axis=1)
  my_calculations_df["sum_factor"] = (my_calculations_df["cosine_score"]*my_calculations_df["non-toxicity_score"])/(my_calculations_df["perplexity_score"]*0.7)
  return my_calculations_df


ci = get_civil_sentence('what are you doing you moron', good_sentences)

what are you doing you moron [['WP', 'VBP', 'PRP', 'VBG', 'PRP', 'BW']]


Downloading:   0%|          | 0.00/693 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/329M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/355 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]


and what are you doing ?


  0%|          | 0/6 [00:00<?, ?it/s]


l ? what are you doing you ?


  0%|          | 0/8 [00:00<?, ?it/s]


In [ ]:
ci

,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
0,what are you doing you moron,what are you not doing ?,0.733778,0.998460,66.786530,0.015671
1,what are you doing you moron,and what are you doing you ?,0.801338,0.998317,119.671608,0.009550
2,what are you doing you moron,and what are you doing you what are you do ?,0.795365,0.998356,60.407906,0.018778
3,what are you doing you moron,what are you ? what are you doing you ?,0.828650,0.995775,41.676247,0.028284
4,what are you doing you moron,i you ? what are you doing you ?,0.797977,0.976912,114.897614,0.009693
5,what are you doing you moron,what are you doing you ?,0.811312,0.998336,98.762985,0.011716
6,what are you doing you moron,what are you doing what are you ?,0.819003,0.999013,37.464668,0.031199
7,what are you doing you moron,and what are you doing ?,0.792343,0.999098,67.948044,0.016644
8,what are you doing you moron,l ? what are you doing you ?,0.778220,0.998338,165.824738,0.006693


In [ ]:
ci[ci['sum_factor'] == ci['sum_factor'].max()]

,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
6,what are you doing you moron,what are you doing what are you ?,0.819003,0.999013,37.464668,0.031199


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
bleu_score = []

In [ ]:
reference = ['I dont think your advice is needed because I am not as dumb as you are'.split()]
candidate = 'you do not think'.split()
score = sentence_bleu(reference, candidate)
bleu_score.append(score)

In [ ]:
ten_bad_sent = [
    "What the fuck are you doing morons",
    "what the hell is wrong with you moron",
    # "Come here you stupid fuck",
    # "Your ass is fired, idiot",
    "You dog-faced, small-cocked, hypocrite bastard son of a weasel and a whore",
    "Your content is completely shit and worthless",
    "I really think you should go fuck yourself",
    "Please leave me alone you racist motherfucker",
    "I really think you are completely worthless asshole",
    # "What I feel is nothing of your concern retard",
    "I dont think your advice is needed because I am not as dumb as you are"
]
outputs = []
for x in ten_bad_sent:
  outputs.append(get_civil_sentence(x, good_sentences))

what the fuck are you doing morons [['WP', 'DT', 'BW', 'VBP', 'PRP', 'VBG', 'NNS']]
[['<mask>', 'are', 'you', 'doing', '<mask>', 'the', '<mask>'], ['<mask>', '<mask>', 'are', 'you', '<mask>', '<mask>', 'doing'], ['are', 'you', 'doing', '<mask>', '<mask>', '<mask>'], ['<mask>', 'are', 'you', '<mask>', 'doing', '<mask>'], ['what', 'are', 'you', 'doing', 'morons', '<mask>', '<mask>', '<mask>', '<mask>', '<mask>', 'the', '<mask>'], ['what', 'are', 'you', 'doing', '<mask>', '<mask>', '<mask>', '<mask>', '<mask>', '<mask>', 'the', 'morons'], ['what', '<mask>', 'you', 'doing', 'morons', 'are', '<mask>', '<mask>', '<mask>', '<mask>', 'the', '<mask>'], ['what', '<mask>', 'you', 'doing', '<mask>', 'are', '<mask>', '<mask>', '<mask>', '<mask>', 'the', 'morons'], ['<mask>', 'what', 'are', 'you', 'doing', '<mask>']]
what are you doing in the world


  0%|          | 0/7 [00:00<?, ?it/s]


what are you the ons doing


  0%|          | 0/7 [00:00<?, ?it/s]


are you doing mor ons ?


...

  0%|          | 0/3 [00:00<?, ?it/s]


i are not


  0%|          | 0/3 [00:00<?, ?it/s]


you are not


  0%|          | 0/3 [00:00<?, ?it/s]


In [ ]:
outputs[7].to_csv("/content/someop.csv")

In [ ]:
from IPython.display import display
for x in outputs:
  ci = x
  display(ci[ci['sum_factor'] == ci['sum_factor'].max()])

,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
0,what the fuck are you doing morons,what are you doing in the world,0.619279,0.997477,14.56428,0.06059


,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
2,what the hell is wrong with you moron,what is wrong with you,0.882624,0.968279,9.844369,0.12402


,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
15,you dog faced small cocked hypocrite bastard s...,is a weasel,0.595887,0.712668,193.025467,0.003143
18,you dog faced small cocked hypocrite bastard s...,is a weasel,0.595887,0.712668,193.025467,0.003143
21,you dog faced small cocked hypocrite bastard s...,is a weasel,0.595887,0.712668,193.025467,0.003143
27,you dog faced small cocked hypocrite bastard s...,is a weasel,0.595887,0.712668,193.025467,0.003143
30,you dog faced small cocked hypocrite bastard s...,is a weasel,0.595887,0.712668,193.025467,0.003143
51,you dog faced small cocked hypocrite bastard s...,is a weasel,0.595887,0.712668,193.025467,0.003143
54,you dog faced small cocked hypocrite bastard s...,is a weasel,0.595887,0.712668,193.025467,0.003143
57,you dog faced small cocked hypocrite bastard s...,is a weasel,0.595887,0.712668,193.025467,0.003143


,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
10,your content is completely shit and worthless,it is not and it is completely worthless and w...,0.939383,0.855655,78.516769,0.014624


,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
8,i really think you should go fuck yourself,you really think,0.495617,0.999065,65.8564,0.010741
11,i really think you should go fuck yourself,you really think,0.495617,0.999065,65.8564,0.010741
14,i really think you should go fuck yourself,you really think,0.495617,0.999065,65.8564,0.010741
17,i really think you should go fuck yourself,you really think,0.495617,0.999065,65.8564,0.010741


,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
0,please leave me alone you racist motherfucker,please leave me alone !,0.658816,0.978424,105.171158,0.008756


,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
1,i really think you are completely worthless as...,i you really you really think i are completely...,0.97665,0.905405,290.845703,0.004343


,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
11,i do not think your advice is needed because i...,you are not,0.42888,0.967873,19.792841,0.02996
23,i do not think your advice is needed because i...,you are not,0.42888,0.967873,19.792841,0.02996


In [ ]:
outputs[7]

,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
0,i do not think your advice is needed because i...,i do as,0.285429,0.998004,1172.177490,0.000347
1,i do not think your advice is needed because i...,you do as,0.302152,0.992448,216.000534,0.001983
2,i do not think your advice is needed because i...,i do not,0.424804,0.998442,343.764465,0.001763
3,i do not think your advice is needed because i...,you do not,0.439451,0.986900,21.197674,0.029228
4,i do not think your advice is needed because i...,i am as,0.312888,0.998711,976.879822,0.000457
5,i do not think your advice is needed because i...,you am as,0.318382,0.977423,1120.519043,0.000397
6,i do not think your advice is needed because i...,i am not,0.414865,0.998168,219.845139,0.002691
7,i do not think your advice is needed because i...,you am not,0.431526,0.964755,271.069061,0.002194
8,i do not think your advice is needed because i...,i are as,0.315823,0.990903,558.777222,0.000800
9,i do not think your advice is needed because i...,you are as,0.322398,0.594131,145.894867,0.001876


In [ ]:
from nltk.translate.bleu_score import sentence_bleu